In [ ]:
#Импортируем библиотеки
import pandas as pd
import numpy as np
import pyodbc
import os
import re
from datetime import date, time, timedelta, datetime
import datetime as dt

In [ ]:
#Задаем функцию для выгрузки данных
def load_from_db(file, sql_request = ''):
    if sql_request:
        sql_request = re.sub(r'\s+', ' ', sql_request)
        sql_request = re.sub(r'\t+', ' ', sql_request)
        print(sql_request)
        
        sql_time = datetime.now()
        print(datetime.now())
        df = pd.read_sql_query(sql_request, pyodbc.connect('DSN=;UID=;PWD=', autocommit=True))
        sql_time = datetime.now() - sql_time
        print(sql_time)
        f = open('Logs_sql_request.txt', 'a')
        f.write(str(sql_time) +' rows='+str(df.shape[0])+' '+sql_request+'\n')
        f.close()
        df.to_csv(file+".csv",index=False,sep=';')
        print(datetime.now())

    return df


**Часть 1**


По ТК из файла выгружаем информацию о клиентах ,чеках, продажах. Цель - найти перечесения клиентов внутри кластера в разрезе группы закупок.

In [ ]:
#Читаем файл со списком ТК
tk = pd.read_excel(r'C:\Users\olga.e.fadeeva\Desktop\Выгрузки\Список ТК.xlsx')

In [ ]:
#Список ТК для запроса
tk_list = tuple(tk['ТК'].tolist())

In [ ]:
#Задаем границы периода
start = '20210524'
end = '20210725' 

In [ ]:
#Запрос
sql = '''
SELECT  t_mat.`/bic/agrsaldir` direction,
        t_mat.`/bic/apur_grp` grup,
        tt1.`/bic/client` client,
        tt1.plant TK,
        COUNT(distinct tt1.check_id) check_count,
        SUM(tt1.rtsaexc) sales,
        cl_ch.cl_checks total_checks
        
        from rdw.rec_itm tt1
        
join rdw.bi0_pmaterial t_mat on tt1.material=t_mat.material

join (select `/bic/client`,
             count(distinct check_id) cl_checks
            from rdw.rec_itm where cast(plant as int) in {0} 
            and calday between {1} and {2}  
            group by `/bic/client`) cl_ch
            on tt1.`/bic/client`=cl_ch.`/bic/client`
        
where cast(tt1.plant as int) in {0} 
and tt1.calday between {1} and {2}    
 group by t_mat.`/bic/agrsaldir`,
        t_mat.`/bic/apur_grp`,
         tt1.`/bic/client`,
         cl_ch.cl_checks,
          tt1.plant
    
'''.format(tk_list, start, end)

In [ ]:
df1 = load_from_db("tk_res_1", sql)

In [ ]:
df1.head()

In [ ]:
df_for_analysis.shape

In [ ]:
#Убираем клиентов с 1 чеком
df_for_analysis = df1.query('total_checks>1')

In [ ]:
df_for_analysis['tk'] = df_for_analysis['tk'].astype('int')

In [ ]:
cluster6 = tuple(tk.query('Cluster=="cluster 6"')['ТК'].tolist())

Пример на 1 группе закупок:

In [ ]:
df_P01 = df_for_analysis.query('grup=="P01" & tk in @cluster6')

In [ ]:
#Смотрим пересечения
df_P01_merged = df_P01.merge(df_P01, on='client')
res_P01 = pd.crosstab(df_P01_merged.tk_x, df_P01_merged.tk_y)

In [ ]:
res_P01.head()

Общее количество клиентов каждого ТК - сохраняем в файл (понадобится для расчета долей)

In [ ]:
clients_number_tk = df_for_analysis.groupby('tk').client.nunique()

In [ ]:
clients_number_tk.to_excel(r'C:\Users\olga.e.fadeeva\Desktop\Выгрузки\clients_number_tk.xlsx')

In [ ]:
grp_list = tuple(df_for_analysis.grp.unique().tolist())

Выгружаем данные для кластера

In [ ]:
for grp in grp_list:
    df_gz = df_for_analysis.query('grup==@grp & tk in @cluster6')
    df_gz1 = df_gz.merge(df_gz, on='client')
    pd.crosstab(df_gz1.tk_x, df_gz1.tk_y).to_excel(r'C:\Users\olga.e.fadeeva\Desktop\Выгрузки\{0}.xlsx'.format(grp))

Объединяем данные из файлов в 1 датафрейм

In [ ]:
Path=r"C:\Users\olga.e.fadeeva\Desktop\Выгрузки\1"
files_list = os.listdir(path=Path)

In [ ]:
df = []
for file in files_list:
    df_buf = pd.read_excel(f"{Path}/"+file)
    df_buf['grp'] = file[:3]
    df.append(df_buf)
    
df = pd.concat(df, ignore_index=True)

In [ ]:
df.fillna(0)

In [ ]:
df.to_excel(r'C:\Users\olga.e.fadeeva\Desktop\Выгрузки\cluster6.xlsx')

**Часть 2**

Выявляем пересечения клиентов между всеми ТК СПб и ЛО, без групп закупок.

In [ ]:
spb_LO = pd.read_excel(r'C:\Users\olga.e.fadeeva\Desktop\Выгрузки\tk_spb_lo.xlsx')

In [ ]:
#Список ТК для запроса
spb_list = tuple(spb_LO['tk'].tolist())

In [ ]:
sql = '''
SELECT  tt1.`/bic/client` client,
        tt1.plant TK,
        COUNT(distinct tt1.check_id) check_count,
        SUM(tt1.rtsaexc) sales,
        cl_ch.cl_checks total_checks
        
        from rdw.rec_itm tt1

join (select `/bic/client`,
             count(distinct check_id) cl_checks
            from rdw.rec_itm where cast(plant as int) in {0} 
            and calday between {1} and {2}
            group by `/bic/client`) cl_ch
            on tt1.`/bic/client`=cl_ch.`/bic/client`
        
where cast(tt1.plant as int) in {0} 
and tt1.calday between {1} and {2}    
 group by tt1.`/bic/client`,
         cl_ch.cl_checks,
          tt1.plant
    
'''.format(spb_list, start, end)

In [ ]:
df_spb = load_from_db("spb_lo", sql)

In [ ]:
#Убираем клиентов с 1 чеком
df_spb_filtered = df_spb.query('total_checks>1')

In [ ]:
#Смотрим пересечения
df_spb_merged = df_spb_filtered.merge(df_spb_filtered, on='client')
res_spb = pd.crosstab(df_spb_merged.tk_x, df_spb_merged.tk_y)

In [ ]:
res_spb.head()

In [ ]:
res_spb.to_excel(r'C:\Users\olga.e.fadeeva\Desktop\Выгрузки\res_spb.xlsx')

In [ ]:
#Общее количество клиентов каждого ТК
clients_number_tk_spb = df_spb_filtered.groupby('tk').client.nunique()

In [ ]:
clients_number_tk_spb.to_excel(r'C:\Users\olga.e.fadeeva\Desktop\Выгрузки\total_clients_spb.xlsx')